In [ ]:
# 클라이언트 모드 - 클러스터 밖 / 클러스터 모드 - 클러스터 안  

In [6]:
import findspark
findspark.init()
findspark.find()

'C:\\bigdata\\spark-2.4.5-bin-hadoop2.7'

In [7]:
import pyspark
findspark.find()

'C:\\bigdata\\spark-2.4.5-bin-hadoop2.7'

### spark 세션 생성 위해 컴파일 진행

In [23]:
from pyspark import SparkContext, SparkConf
# SparkContext 객체 : Spark 에 클러스터에 엑세스하는 방법과 위치 알려줌 
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [22]:
# 세션 종료 
sc.stop()

### RDD 만들기 

In [44]:
data = [1,2,3,4,5]
data

[1, 2, 3, 4, 5]

In [45]:
rdd = sc.parallelize(data, 4)
rdd

ParallelCollectionRDD[37] at parallelize at PythonRDD.scala:195

In [46]:
sc.defaultParallelism

2

In [47]:
rdd1 = rdd.map(lambda x : x*2)
# transform : - 결과 변경 

In [48]:
rdd1.collect()
# 액션 : 드라이브 프로세스로 들어옴 - 결과 요청

[2, 4, 6, 8, 10]

In [49]:
rdd2 = rdd.filter(lambda x : x%2 == 0)

In [50]:
rdd2.collect()

[2, 4]

In [51]:
rdd3 = sc.parallelize([1,4,2,2,3])
rdd3.distinct() # 중복 제거 

PythonRDD[45] at RDD at PythonRDD.scala:53

In [52]:
rdd3.distinct().collect() # 중복 값 빠짐 

[4, 2, 1, 3]

In [53]:
rdd4 = sc.parallelize([1,2,3])
rdd4.map(lambda x : [x, x+5]).collect() # map() : 리스트 안에 리스트 

[[1, 6], [2, 7], [3, 8]]

In [57]:
rdd4.flatMap(lambda x : [x, x+5]).collect() # flatMap() : 인자만 

[1, 6, 2, 7, 3, 8]

### Action 
reduce(func) : 처리 결과   
take(n) : 일부만 갖고오기   
collect() : 전체 갖고오기, 결과 반환하는 것 * 큰 데이터 쓸 때 조심하기! 드라이브로 다 들어온다      
takeOrdereded(n, key=func) : 몇 개 가져올 지 지정 오름차순,내림차순 지정   


In [58]:
rdd = sc.parallelize([1,2,3])
rdd.reduce(lambda a,b : a*b)

6

In [59]:
rdd.take(2)

[1, 2]

In [60]:
rdd.collect()

[1, 2, 3]

In [64]:
rdd5 = sc.parallelize([5,3,1,2])
rdd5.takeOrdered(3, lambda s : -1 * s)

[5, 3, 2]

### 스파크 데이터 타입 

- ByteType : int, long - 128 ~ 127 사이의 값 
- ShortType : int, long - 32768 ~ 32767 사이의 값 
- IntegerType : int, long 2바이트 크기 
- LongType : long 8바이트 크기 
- FloatType : float 4바이트 크기 
- DoubleType : float 
- DecimalType : decimal.Decimal 
- StringType : string 
- BinaryType : bytearray
- BooleanType : bool 
- TimestampType : datetime.datetime 
- DateType : datetime.date 
- ArrayType : list, tuple, array

### DataFrame

Row 타입의 레코드(테이블의 로우 같은)와 각 레코드에 수행할 연산 표현식을  
나타내는 여러 컬럼(스프레드시트의 컬럼 같은)으로 구성됩니다.  
스키마는 각 컬럼명과 데이터 타입을 정의 

In [65]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json")

In [66]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [67]:
spark.read.format("json").load("./data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [69]:
'''
StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),
StructField(ORIGIN_COUNTRY_NAME,StringType,true),
StructField(count,LongType,true)))
'''

'\nStructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),\nStructField(ORIGIN_COUNTRY_NAME,StringType,true),\nStructField(count,LongType,true)))\n'

In [70]:
from pyspark.sql.types import StructField, StructType, StringType, LongType 

In [72]:
myManualSchema = StructType([
    StructField("StructField(DEST_COUNTRY_NAME",StringType(),True), 
    StructField("ORIGIN_COUNTRY_NAME",StringType(),True), 
    StructField("count",LongType(),False, metadata={"hello":"world"}), 
    ])
df = spark.read.format("json").schema(myManualSchema).load("./data/flight-data/json/2015-summary.json")

In [73]:
df.printSchema()

root
 |-- StructField(DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [74]:
df.take(3)

[Row(StructField(DEST_COUNTRY_NAME=None, ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(StructField(DEST_COUNTRY_NAME=None, ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(StructField(DEST_COUNTRY_NAME=None, ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [76]:
from pyspark.sql.functions import col, column

In [77]:
col("someColumnName")
column("someColumnName")

Column<b'someColumnName'>

In [82]:
df.columns

['StructField(DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [83]:
from pyspark.sql import Row 
myRow = Row("Hello", None, 1, False)

In [84]:
myRow[0]

'Hello'

In [86]:
myRow[2]

1

In [87]:
myRow[3]

False

In [88]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json")

In [89]:
df.createOrReplaceTempView('dfTable')

In [90]:
myManualSchema = StructType([
    StructField("some",StringType(),True), 
    StructField("col",StringType(),True), 
    StructField("names",LongType(),False) 
    ])
myRow = Row("Hello", None, 1)
myDF = spark.createDataFrame([myRow], myManualSchema)

In [91]:
myDF.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



In [94]:
type(myDF) # spark 의 데이터프레임이다 (pandas의 데이터프레임이 아니다 )

pyspark.sql.dataframe.DataFrame

In [95]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



'''SQL  
SELECT DEST_COUNTRY_NAME FROM dfTable LIMIT 2   
'''

In [96]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show()

+--------------------+-------------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+--------------------+-------------------+
|       United States|            Romania|
|       United States|            Croatia|
|       United States|            Ireland|
|               Egypt|      United States|
|       United States|              India|
|       United States|          Singapore|
|       United States|            Grenada|
|          Costa Rica|      United States|
|             Senegal|      United States|
|             Moldova|      United States|
|       United States|       Sint Maarten|
|       United States|   Marshall Islands|
|              Guyana|      United States|
|               Malta|      United States|
|            Anguilla|      United States|
|             Bolivia|      United States|
|       United States|           Paraguay|
|             Algeria|      United States|
|Turks and Caicos ...|      United States|
|       United States|          Gibraltar|
+----------

In [ ]:
'''SQL  
SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME FROM dfTable LIMIT 2   
'''

In [104]:
df.selectExpr(
    "*", # 모든 원본 컬럼
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry"
).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [ ]:
'''SQL
SELECT *, (DEST_COUNTRY_NAME" = "ORIGIN_COUNTRY_NAME) as withinCountry
FROM dfTable 
LIMIT 2 
'''

In [101]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)
# selectExpr : 데티어 테이블에 SQL을 실행하는 것처럼 DataFrame에서도 SQL를 사용할 수 있음

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [102]:
'''SQL
SELECT avg(count), count(distinct(DEST_COUNTRY_NAME))
FROM dfTable 
LIMIT 2 
'''

'SQL\nSELECT avg(count), count(distinct(DEST_COUNTRY_NAME))\nFROM dfTable \nLIMIT 2 \n'